### Es 1
Hai a disposizione un file `data.csv` contenente dati mensili di passeggeri con due colonne:

- `date`: data in formato `YYYY-MM` (mese/anno)
- `passengers`: numero di passeggeri per quel mese


Costruisci un modello di **regressione polinomiale** che approssima l’andamento del numero di passeggeri nel tempo.

1. Carica il dataset.
2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.
3. Applica una regressione polinomiale (grado a tua scelta).
4. Calcola l’RMSE tra i valori reali e quelli predetti.
5. Visualizza i dati reali e la curva stimata con Plotly.

In [63]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
import plotly.graph_objects as go
import scipy
import math
from sklearn.metrics import r2_score


dati=pd.read_csv("/Users/michele/Downloads/data.csv")

df=pd.DataFrame(dati)
df["passengers"].dropna(inplace=True)

df["mese_numerico"]= range(len(df))
x=df[['mese_numerico']].values

y=df['passengers'].values
Xtrain, Xtest, ytrain, ytest =train_test_split(x,y,train_size=104, shuffle=False)

model_poly2 = make_pipeline(PolynomialFeatures(3), LinearRegression())
xpoly=model_poly2.fit(Xtrain, ytrain)
y_pred_poly2 = model_poly2.predict(Xtest)



rmse =  root_mean_squared_error(ytest, y_pred_poly2)
print("l'rmse vale " ,rmse)

fig =px.scatter(data_frame= df, x= "mese_numerico", y="passengers")

fig.add_scatter(x = df["mese_numerico"]+len(Xtrain), y=y_pred_poly2)











l'rmse vale  71.3344305597801


### Es2. 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

In [62]:
import dash                      
from dash import dcc, html      
from dash.dependencies import Input, Output
import plotly.express as px     
import pandas as pd  
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import _polynomial

app = dash.Dash(__name__)  

app.layout = html.Div([
    html.H3("seleziona"), 

    
    dcc.Slider(
        id='slider-numero',       
        min=1,
        max=10,
        step=1,
        value=1,                   
        marks={i: str(i) for i in range(1, 11)}  
    ),

   
    dcc.Graph(id='scatter-plot')
    
])

@app.callback(
    dash.dependencies.Output('scatter-plot', 'figure'),  
    [dash.dependencies.Input('slider-numero', 'value')]
)
def update_figure(valori):
    
    X=np.random.uniform(-3,3,100)
    noise=np.random.normal(0,2,100)
    y=X*X*X-X+noise
    df=pd.DataFrame()
    df["X"]=X
    df["y"]=y
    variabx=df[["X"]].values
    variaby=df["y"]
    
    polinomial=make_pipeline(PolynomialFeatures(valori), LinearRegression())
    polinomial.fit(variabx,variaby)
    yprev=polinomial.predict(variabx)
   
    fig = px.scatter(x=df['X'],y=df["y"])
    fig.add_scatter(x=df['X'],y=yprev)
    fig.update_layout(title="grafico che mostra i punti e la regressione", xaxis_title="X", yaxis_title="Y")
    return fig



if __name__ == '__main__':
    app.run(debug=True)
